## <b><font color='darkblue'>Model Context Protocol Tools</font></b>
([source](https://google.github.io/adk-docs/tools/mcp-tools/)) <b><font size='3ptx'>This guide walks you through two ways of integrating Model Context Protocol (MCP) with ADK.</font></b>

In [8]:
from IPython.display import display, Markdown, Latex

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

### <b><font color='darkgreen'>What is Model Context Protocol (MCP)?</font></b>
([source](https://google.github.io/adk-docs/tools/mcp-tools/#what-is-model-context-protocol-mcp)) <font size='3ptx'><b>The [Model Context Protocol](https://modelcontextprotocol.io/docs/getting-started/intro) (MCP) is an open standard designed to standardize how Large Language Models (LLMs) like Gemini and Claude communicate with external applications, data sources, and tools.</b>  Think of it as a universal connection mechanism that simplifies how LLMs obtain context, execute actions, and interact with various systems.</font>

MCP follows a client-server architecture, defining how **data** (<font color='brown'>resources</font>), **interactive templates** (<font color='brown'>prompts</font>), and **actionable functions** (<font color='brown'>tools</font>) are exposed by an MCP server and consumed by an MCP client (<font color='brown'>which could be an LLM host application or an AI agent</font>).

This guide covers two primary integration patterns:
1. <b><font size='3ptx'>Using Existing MCP Servers within ADK</font></b>: An ADK agent acts as an MCP client, leveraging tools provided by external MCP servers.
2. <b><font size='3ptx'>Exposing ADK Tools via an MCP Server</font></b>: Building an MCP server that wraps ADK tools, making them accessible to any MCP client.

### <b><font color='darkgreen'>Prerequisites</font></b>
Before you begin, ensure you have the following set up:
* <b><font size='3ptx'>Set up ADK</font></b>: Follow the standard ADK [setup instructions](https://google.github.io/adk-docs/get-started/quickstart/#venv-install) in the quickstart.
* <b><font size='3ptx'>Setup `Node.js` and `npx`</font></b>: Many community MCP servers are distributed as `Node.js` packages and run using npx. Install `Node.js` (which includes npx) if you haven't already. For details, see https://nodejs.org/en.
* <b><font size='3ptx'>Verify Installations</font></b>: Confirm adk and npx are in your PATH within the activated virtual environment:

In [1]:
!which adk

/usr/local/google/home/johnkclee/Github/ml_articles/env/bin/adk


In [2]:
!which npx

/usr/local/google/home/johnkclee/.nvm/versions/node/v22.16.0/bin/npx


### <b><font color='darkgreen'>1. Using MCP servers with ADK agents (ADK as an MCP client) in `adk web`</font></b>
([source](https://google.github.io/adk-docs/tools/mcp-tools/#1-using-mcp-servers-with-adk-agents-adk-as-an-mcp-client-in-adk-web)) <font size='3ptx'><b>This section demonstrates how to integrate tools from external MCP (Model Context Protocol) servers into your ADK agents.</b> This is the most common integration pattern when your ADK agent needs to use capabilities provided by an existing service that exposes an MCP interface.</font>

<font size='3ptx'>You will see how the <b><font color='blue'>MCPToolset</font></b> class can be directly added to your agent's tools list, enabling seamless connection to an MCP server, discovery of its tools, and making them available for your agent to use</font>. These examples primarily focus on interactions within the `adk web` development environment.

#### <b>`MCPToolset` class</b>
<font size='3ptx'>The <b><font color='blue'>MCPToolset</font></b> class is ADK's primary mechanism for integrating tools from an MCP server</font>. When you include an <b><font color='blue'>MCPToolset</font></b> instance in your agent's `tools` list, it automatically handles the interaction with the specified MCP server. Here's how it works:

1. <b><font size='3ptx'>Connection Management</font></b>: On initialization, <b><font color='blue'>MCPToolset</font></b> establishes and manages the connection to the MCP server. This can be a local server process (<font color='brown'>using `StdioConnectionParams` for communication over standard input/output</font>) or a remote server (<font color='brown'>using `SseConnectionParams` for Server-Sent Events</font>). The toolset also handles the graceful shutdown of this connection when the agent or application terminates.
2. <b><font size='3ptx'>Tool Discovery & Adaptation</font></b>: Once connected, <b><font color='blue'>MCPToolset</font></b> queries the MCP server for its available tools (<font color='brown'>via the `list_tools` MCP method</font>). It then converts the schemas of these discovered MCP tools into ADK-compatible <b><font color='blue'>BaseTool</font></b> instances.
3. <b><font size='3ptx'>Exposure to Agent</font></b>: These adapted tools are then made available to your `LlmAgent` as if they were native ADK tools.
4. <b><font size='3ptx'>Proxying Tool Calls</font></b>: When your `LlmAgent` decides to use one of these tools, <b><font color='blue'>MCPToolset</font></b> transparently proxies the call (<font color='brown'>using the `call_tool` MCP method</font>) to the MCP server, sends the necessary arguments, and returns the server's response back to the agent.
5. <b><font size='3ptx'>Filtering (Optional)</font></b>: You can use the tool_filter parameter when creating an MCPToolset to select a specific subset of tools from the MCP server, rather than exposing all of them to your agent.

The following examples demonstrate how to use <b><font color='blue'>MCPToolset</font></b> within the adk web development environment. For scenarios where you need more fine-grained control over the MCP connection lifecycle or are not using `adk web`, refer to the "**Using MCP Tools in your own Agent out of `adk web`**" section later in this page

#### <b>Example 1: File System MCP Server</b>
This example demonstrates connecting to a local MCP server that provides file system operations.

##### <b>Step 1</b>: Define your Agent with <b><font color='blue'>MCPToolset</font></b>
Create an <font color='olive'>agent.py</font> file (<font color='brown'>e.g., in</font> <font color='olive'>./mcp_examples/mcp_filesystem/agent.py</font>). The <b><font color='blue'>MCPToolset</font></b> is instantiated directly within the tools list of your `LlmAgent`:

* <b><font color='olive'>./mcp_examples/mcp_filesystem/agent.py</font></b>
```python
import os # Required for path operations
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioConnectionParams, StdioServerParameters

# It's good practice to define paths dynamically if possible,
# or ensure the user understands the need for an ABSOLUTE path.
# For this example, we'll construct a path relative to this file,
# assuming '/path/to/your/folder' is in the same directory as agent.py.
# REPLACE THIS with an actual absolute path if needed for your setup.
TARGET_FOLDER_PATH = os.path.join(os.path.dirname(os.path.abspath(__file__)), "/path/to/your/folder")
# Ensure TARGET_FOLDER_PATH is an absolute path for the MCP server.
# If you created ./mcp_examples/mcp_filesystem/your_folder,

root_agent = LlmAgent(
    model='gemini-2.0-flash',
    name='filesystem_assistant_agent',
    instruction='Help the user manage their files. You can list files, read files, etc.',
    tools=[
        MCPToolset(
            connection_params=StdioConnectionParams(
                server_params = StdioServerParameters(
                    command='npx',
                    args=[
                        "-y",  # Argument for npx to auto-confirm install
                        "@modelcontextprotocol/server-filesystem",
                        # IMPORTANT: This MUST be an ABSOLUTE path to a folder the
                        # npx process can access.
                        # Replace with a valid absolute path on your system.
                        # For example: "/Users/youruser/accessible_mcp_files"
                        # or use a dynamically constructed absolute path:
                        os.path.abspath(TARGET_FOLDER_PATH),
                    ],
                ),
            ),
            # Optional: Filter which tools from the MCP server are exposed
            # tool_filter=['list_directory', 'read_file']
        )
    ],
)
```

##### <b>Step 2</b>: Create an <font color='olive'>\_\_init__.py</font> file
Ensure you have an `__init__.py` in the same directory as `agent.py` to make it a discoverable Python package for ADK:
```python
# ./mcp_examples/mcp_filesystem/__init__.py
from . import agent
```

##### <b>Step 3</b>: Run `adk web` and Interact
Navigate to the parent directory of `mcp_filesystem` (e.g., `mcp_examples`) in your terminal and run:
```shell
$ ls
mcp_filesystem

$ adk web
```

<b><font color='orange'>Notes.</font></b>
> When hitting the `_make_subprocess_transport NotImplementedError`, consider using a`dk web --no-reload` instead.

Once the ADK Web UI loads in your browser:
1. Select the `mcp_filesystem` from the agent dropdown.
2. Try prompts like:
    - "`List files in the current directory.`"
    - "`Can you read the file named sample.txt?`" (<font color='brown'>assuming you created it in `TARGET_FOLDER_PATH`</font>).
    - "`What is the content of another_file.md?`"


You should see the agent interacting with the MCP file system server, and the server's responses (<font color='brown'>file listings, file content</font>) relayed through the agent. The adk web console (<font color='brown'>terminal where you ran the command</font>) might also show logs from the npx process if it outputs to stderr.
![ui](images/tools_mcp_1.png)

#### <b>Example 2: Google Maps MCP Server</b>
This example demonstrates connecting to the Google Maps MCP server. ([more](https://google.github.io/adk-docs/tools/mcp-tools/#example-2-google-maps-mcp-server))

### <b><font color='darkgreen'>2. Building an MCP server with ADK tools (MCP server exposing ADK)</font></b>
([source](https://google.github.io/adk-docs/tools/mcp-tools/#2-building-an-mcp-server-with-adk-tools-mcp-server-exposing-adk)) <font size='3ptx'><b>This pattern allows you to wrap existing ADK tools and make them available to any standard MCP client application. </b>The example in this section exposes the ADK `load_web_page` tool through a custom-built MCP server.</font>

#### <b>Summary of steps</b>
You will create a **standard Python MCP server application** using the `mcp` library. Within this server, you will:

1. <b><font size='3ptx'>Instantiate</font></b> the ADK tool(s) you want to expose (<font color='brown'>e.g., `FunctionTool(load_web_page)`</font>).
2. <b><font size='3ptx'>Implement</font></b> the MCP server's `@app.list_tools()` handler to advertise the ADK tool(s). This involves converting the ADK tool definition to the MCP schema using the `adk_to_mcp_tool_type` utility from <b><font color='blue'>google.adk.tools.mcp_tool.conversion_utils</font></b>.
3. <b><font size='3ptx'>Implement</font></b> the MCP server's `@app.call_tool()` handler. This handler will:
     - Receive tool call requests from MCP clients.
     - Identify if the request targets one of your wrapped ADK tools.
     - Execute the ADK tool's `.run_async()` method.
     - Format the ADK tool's result into an MCP-compliant response (e.g., `mcp.types.TextContent`).

#### <b>Prerequisites</b>
Install the MCP server library in the same Python environment as your ADK installation:

In [6]:
#!pip install mcp
!pip freeze | grep -P "(mcp=)"

fastmcp==2.11.2
mcp==1.12.0


#### <b>Step 1: Create the MCP Server Script</b>
Create a new Python file for your MCP server, for example, <font color='olive'>my_adk_mcp_server.py</font>.

#### <b>Step 2: Implement the Server Logic</b>
Add the following code to `my_adk_mcp_server.py`. This script sets up an MCP server that exposes the ADK `load_web_page` tool.

In [10]:
show_source_code('my_adk_mcp_server.py')


```python
#!/usr/bin/env python
import asyncio
import json
import os
from dotenv import load_dotenv

# MCP Server Imports
from mcp import types as mcp_types # Use alias to avoid conflict
from mcp.server.lowlevel import Server, NotificationOptions
from mcp.server.models import InitializationOptions
import mcp.server.stdio # For running as a stdio server

# ADK Tool Imports
from google.adk.tools.function_tool import FunctionTool
from google.adk.tools.load_web_page import load_web_page # Example ADK tool
# ADK <-> MCP Conversion Utility
from google.adk.tools.mcp_tool.conversion_utils import adk_to_mcp_tool_type

# --- Load Environment Variables (If ADK tools need them, e.g., API keys) ---
_ = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file


# --- Prepare the ADK Tool ---
# Instantiate the ADK tool you want to expose.
# This tool will be wrapped and called by the MCP server.
print("Initializing ADK load_web_page tool...")
adk_tool_to_expose = FunctionTool(load_web_page)
print(f"ADK tool '{adk_tool_to_expose.name}' initialized and ready to be exposed via MCP.")
# --- End ADK Tool Prep ---


# --- MCP Server Setup ---
print("Creating MCP Server instance...")
# Create a named MCP Server instance using the mcp.server library
app = Server("adk-tool-exposing-mcp-server")


# Implement the MCP server's handler to list available tools
@app.list_tools()
async def list_mcp_tools() -> list[mcp_types.Tool]:
  """MCP handler to list tools this server exposes."""
  print("MCP Server: Received list_tools request.")
  # Convert the ADK tool's definition to the MCP Tool schema format
  mcp_tool_schema = adk_to_mcp_tool_type(adk_tool_to_expose)
  print(f"MCP Server: Advertising tool: {mcp_tool_schema.name}")
  return [mcp_tool_schema]


# Implement the MCP server's handler to execute a tool call
@app.call_tool()
async def call_mcp_tool(
    name: str, arguments: dict
) -> list[mcp_types.Content]: # MCP uses mcp_types.Content
  """MCP handler to execute a tool call requested by an MCP client."""
  print(f"MCP Server: Received call_tool request for '{name}' with args: {arguments}")

  # Check if the requested tool name matches our wrapped ADK tool
  if name == adk_tool_to_expose.name:
    try:
      # Execute the ADK tool's run_async method.
      # Note: tool_context is None here because this MCP server is
      # running the ADK tool outside of a full ADK Runner invocation.
      # If the ADK tool requires ToolContext features (like state or auth),
      # this direct invocation might need more sophisticated handling.
      adk_tool_response = await adk_tool_to_expose.run_async(
          args=arguments,
          tool_context=None,
      )
      print(f"MCP Server: ADK tool '{name}' executed. Response: {adk_tool_response}")

      # Format the ADK tool's response (often a dict) into an MCP-compliant format.
      # Here, we serialize the response dictionary as a JSON string within TextContent.
      # Adjust formatting based on the ADK tool's output and client needs.
      response_text = json.dumps(adk_tool_response, indent=2)
      # MCP expects a list of mcp_types.Content parts
      return [mcp_types.TextContent(type="text", text=response_text)]

    except Exception as e:
      print(f"MCP Server: Error executing ADK tool '{name}': {e}")
      # Return an error message in MCP format
      error_text = json.dumps({"error": f"Failed to execute tool '{name}': {str(e)}"})
      return [mcp_types.TextContent(type="text", text=error_text)]
  else:
    # Handle calls to unknown tools
    print(f"MCP Server: Tool '{name}' not found/exposed by this server.")
    error_text = json.dumps({"error": f"Tool '{name}' not implemented by this server."})
    return [mcp_types.TextContent(type="text", text=error_text)]


# --- MCP Server Runner ---
async def run_mcp_stdio_server():
  """Runs the MCP server, listening for connections over standard input/output."""
  # Use the stdio_server context manager from the mcp.server.stdio library
  async with mcp.server.stdio.stdio_server() as (read_stream, write_stream):
    print("MCP Stdio Server: Starting handshake with client...")
    await app.run(
        read_stream,
        write_stream,
        InitializationOptions(
            server_name=app.name, # Use the server name defined above
            server_version="0.1.0",
            capabilities=app.get_capabilities(
                # Define server capabilities - consult MCP docs for options
                notification_options=NotificationOptions(),
                experimental_capabilities={},
            ),
        ),
    )
    print("MCP Stdio Server: Run loop finished or client disconnected.")


if __name__ == "__main__":
  print("Launching MCP Server to expose ADK tools via stdio...")
  try:
    asyncio.run(run_mcp_stdio_server())
  except KeyboardInterrupt:
    print("\nMCP Server (stdio) stopped by user.")
  except Exception as e:
    print(f"MCP Server (stdio) encountered an error: {e}")
  finally:
    print("MCP Server (stdio) process exiting.")
# --- End MCP Server ---
```

#### <b>Step 3: Test your Custom MCP Server with an ADK Agent</b>
Now, create an ADK agent that will act as a client to the MCP server you just built. This ADK agent will use <b><font color='blue'>MCPToolset</font></b> to connect to your <font color='olive'>my_adk_mcp_server.py script</font>. Create an `agent.py` (<font color='brown'>e.g., in</font> <font color='olive'>./mcp_examples/mcp_with_wrapped_adk_tool/agent.py</font>):

In [11]:
show_source_code('mcp_examples/mcp_with_wrapped_adk_tool/agent.py')


```python
#!/usr/bin/env python
import os
import pathlib

from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StdioConnectionParams, StdioServerParameters

Path = pathlib.Path

# IMPORTANT: Replace this with the ABSOLUTE path to your my_adk_mcp_server.py script
PATH_TO_YOUR_MCP_SERVER_SCRIPT = str(
    Path(os.path.dirname(os.path.abspath(__file__))) /
    Path('my_adk_mcp_server.py'))


if not os.path.isfile(PATH_TO_YOUR_MCP_SERVER_SCRIPT):
  raise Exception(
      f'MCP server script does not exist: {PATH_TO_YOUR_MCP_SERVER_SCRIPT}')


root_agent = LlmAgent(
    model='gemini-2.0-flash',
    name='web_reader_mcp_client_agent',
    instruction=(
        "Use the 'load_web_page' tool to fetch content from a URL provided by "
        "the user."),
    tools=[
        MCPToolset(
            connection_params=StdioConnectionParams(
                server_params = StdioServerParameters(
                    command='python', # Command to run your MCP server script
                    args=[PATH_TO_YOUR_MCP_SERVER_SCRIPT], # Argument is the path to the script
                )
            )
            # tool_filter=['load_web_page'] # Optional: ensure only specific tools are loaded
        )
    ],
)
```

And an `__init__.py` in the same directory:
```python
from . import agent
```

To run the test:
1. <b><font size='3ptx'>Start your custom MCP server</font></b> (optional, for separate observation): You can run your `my_adk_mcp_server.py` directly in one terminal to see its logs: `python3 /path/to/your/my_adk_mcp_server.py`
2. <b><font size='3ptx'>Run `adk web` for the client agent</font></b>: Navigate to the parent directory of `mcp_with_wrapped_adk_tool` and run:
```shell
$ adk web
```

3. <b><font size='3ptx'>Interact in the ADK Web UI</font></b>:
    - Select the `mcp_with_wrapped_adk_tool`.
    - Try a prompt like: "`Load the content from https://example.com`"

This example demonstrates how ADK tools can be encapsulated within an MCP server, making them accessible to a broader range of MCP-compliant clients, not just ADK agents. Refer to the [documentation](https://modelcontextprotocol.io/quickstart/server#core-mcp-concepts), to try it out with Claude Desktop.

### <b><font color='darkgreen'>Using MCP Tools in your own Agent out of `adk web`</font></b>
([source](https://google.github.io/adk-docs/tools/mcp-tools/#using-mcp-tools-in-your-own-agent-out-of-adk-web)) This section is relevant to you if:
* You are developing your own Agent using ADK
* And, you are NOT using `adk web`,
* And, you are exposing the agent via your own UI

Using MCP Tools requires a different setup than using regular tools, due to the fact that specs for MCP Tools are fetched asynchronously from the MCP Server running remotely, or in another process.

The following example is modified from the "Example 1: File System MCP Server" example above. The main differences are:
1. Your tool and agent are created asynchronously
2. You need to properly manage the exit stack, so that your agents and tools are destructed properly when the connection to MCP Server is closed.

In [12]:
show_source_code('tools_mcp_example1.py')


```python
#!/usr/bin/env python
import pathlib
import os
import asyncio

from dotenv import find_dotenv, load_dotenv
from google.genai import types
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import InMemoryArtifactService # Optional
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, SseConnectionParams, StdioConnectionParams, StdioServerParameters


Path = pathlib.Path

# --- Load Environment Variables (If ADK tools need them, e.g., API keys) ---
_ = load_dotenv(find_dotenv(os.path.expanduser('~/.env'))) # read local .env file


# IMPORTANT: Replace this with the ABSOLUTE path to your my_adk_mcp_server.py script
TARGET_FOLDER_PATH = (
    str(Path(os.path.dirname(os.path.abspath(__file__))) / Path('mcp_examples/mcp_filesystem/test/')))


# --- Step 1: Agent Definition ---
async def get_agent_async():
  """Creates an ADK Agent equipped with tools from the MCP Server."""
  toolset = MCPToolset(
      # Use StdioConnectionParams for local process communication
      connection_params=StdioConnectionParams(
          server_params = StdioServerParameters(
            command='npx', # Command to run the server
            args=["-y",    # Arguments for the command
                "@modelcontextprotocol/server-filesystem",
                TARGET_FOLDER_PATH],
          ),
      ),
      tool_filter=['read_file', 'list_directory'] # Optional: filter specific tools
      # For remote servers, you would use SseConnectionParams instead:
      # connection_params=SseConnectionParams(url="http://remote-server:port/path", headers={...})
  )

  # Use in an agent
  root_agent = LlmAgent(
      model='gemini-2.0-flash', # Adjust model name if needed based on availability
      name='enterprise_assistant',
      instruction='Help user accessing their file systems',
      tools=[toolset], # Provide the MCP tools to the ADK agent
  )
  return root_agent, toolset


# --- Step 2: Main Execution Logic ---
async def async_main():
  session_service = InMemorySessionService()
  # Artifact service might not be needed for this example
  artifacts_service = InMemoryArtifactService()

  session = await session_service.create_session(
      state={}, app_name='mcp_filesystem_app', user_id='user_fs'
  )

  # TODO: Change the query to be relevant to YOUR specified folder.
  # e.g., "list files in the 'documents' subfolder" or "read the file 'notes.txt'"
  query = f"list files in the path {TARGET_FOLDER_PATH}"
  print(f"User Query: '{query}'")
  content = types.Content(role='user', parts=[types.Part(text=query)])

  root_agent, toolset = await get_agent_async()

  runner = Runner(
      app_name='mcp_filesystem_app',
      agent=root_agent,
      artifact_service=artifacts_service, # Optional
      session_service=session_service,
  )

  print("Running agent...")
  events_async = runner.run_async(
      session_id=session.id, user_id=session.user_id, new_message=content
  )

  async for event in events_async:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)

  # Cleanup is handled automatically by the agent framework
  # But you can also manually close if needed:
  print("Closing MCP server connection...")
  await toolset.close()
  print("Cleanup complete.")


if __name__ == '__main__':
  try:
    asyncio.run(async_main())
  except Exception as e:
    print(f"An error occurred: {e}")
```

### <b><font color='darkgreen'>Key considerations</font></b>
When working with MCP and ADK, keep these points in mind:

* <b><font size='3ptx'>Protocol vs. Library</font></b>: MCP is a protocol specification, defining communication rules. ADK is a Python library/framework for building agents. <b><font color='blue'>MCPToolset</font></b> bridges these by implementing the client side of the MCP protocol within the ADK framework. Conversely, building an MCP server in Python requires using the model-context-protocol library.
* <b><font size='3ptx'>ADK Tools vs. MCP Tools</font></b>:
    - ADK Tools (`BaseTool`, `FunctionTool`, `AgentTool`, etc.) are Python objects designed for direct use within the ADK's `LlmAgent` and `Runner`.
    - MCP Tools are capabilities exposed by an MCP Server according to the protocol's schema. <b><font color='blue'>MCPToolset</font></b> makes these look like ADK tools to an `LlmAgent`.
    - [**Langchain**](https://python.langchain.com/docs/integrations/tools/#all-tools)/[**CrewAI**](https://www.crewai.com/) Tools are specific implementations within those libraries, often simple functions or classes, lacking the server/protocol structure of MCP. ADK offers wrappers (LangchainTool, CrewaiTool) for some interoperability.
* <b><font size='3ptx'>Asynchronous nature</font></b>: Both ADK and the MCP Python library are heavily based on the [**asyncio**](https://docs.python.org/3/library/asyncio.html) Python library. Tool implementations and server handlers should generally be async functions.
* <b><font size='3ptx'>Stateful sessions (MCP)</font></b>: MCP establishes stateful, persistent connections between a client and server instance. This differs from typical stateless REST APIs.
    - <b>Deployment</b>: This statefulness can pose challenges for scaling and deployment, especially for remote servers handling many users. The original MCP design often assumed client and server were co-located. Managing these persistent connections requires careful infrastructure considerations (<font color='brown'>e.g., load balancing, session affinity</font>).
    - <b>ADK MCPToolset</b>: Manages this connection lifecycle. The `exit_stack` pattern shown in the examples is crucial for ensuring the connection (<font color='brown'>and potentially the server process</font>) is properly terminated when the ADK agent finishes.

### <b><font color='darkgreen'>Further Resources</font></b>
* [Model Context Protocol Documentation](https://modelcontextprotocol.io/)
* [MCP Specification](https://modelcontextprotocol.io/specification/)
* [MCP Python SDK & Examples](https://github.com/modelcontextprotocol/)